In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [5]:
from sklearn.ensemble import ExtraTreesRegressor

In [6]:
def display_scores(scores):
    print("Scores:",scores)
    print("Mean:",scores.mean())
    print("Standard deviation:",scores.std())

In [133]:
df = pd.read_csv("./kc_house_data.csv")

### date에서 시간을 제거하고 년/월/일만 남겨 int 타입으로 저장

In [134]:
df['date']=df['date'].str.slice(start=0, stop=8)
df['date']=df['date'].astype(int)
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


### 방 개수가 33개인 특성 제거하기

In [136]:
df=df[df['bedrooms']!=33]

### 이상치 제거하기

In [138]:
def remove_outliers(df):
    '''removes entries with z-score above 3 for specific columns'''
    variables = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 
                 'lat', 'long', 'sqft_living15', 'sqft_lot15']
    
    for variable in variables:
        df = df[np.abs(df[variable]-df[variable].mean()) <= (3*df[variable].std())]
        
    return df

In [139]:
redf = remove_outliers(df)

In [140]:
rex = redf.drop(columns=['price','id'], axis=1)
rey = redf["price"].copy()

## ExtraTreeRegressor

In [67]:
reetr = ExtraTreesRegressor(n_estimators=1000, random_state=0)

In [141]:
reetr_scores = cross_val_score(reetr,rex,rey,cv=10,scoring="neg_mean_squared_error")
reetr_rmse = np.sqrt(-reetr_scores)
display_scores(reetr_rmse)

Scores: [ 96413.6823497   96365.26515517  95521.27704632  98546.51522331
  90706.31806613  98521.80442401 103506.71015031 100762.00998123
 100339.46740484 100159.24065305]
Mean: 98084.2290454061
Standard deviation: 3363.3112034333194
